In [25]:
from train import *
from model import *
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset

In [26]:
class AAUSewer(Dataset):
    def __init__(self,split = "train",type = "real"):
        super().__init__()
        self.split = split
        
        self.train_data = []
        self.labels = []

        path = os.path.join(dataDir, "{}_{}.h5".format("{}ing_pointcloud_hdf5".format(split), type))
        print(path)
        with h5py.File(path, 'r') as hdf:          
            if split == "train":
                partitions = ["Training"]
            else:
                partitions = ["Testing"]

            for partition in partitions:
                self.train_data = np.asarray(hdf[f'{partition}/PointClouds'][:])
                self.labels = np.asarray(hdf[f'{partition}/Labels'][:])
      

    def __len__(self):return self.labels.shape[0]

    def __getitem__(self,index):
        labels = self.labels[index]
        if labels > 1: labels = 1
        return self.train_data[index],labels

In [27]:

import argparse
model = PointNetCls(2)
#model = torch.load("point_net.ckpt")

aau_syn_train =  aau_syn = AAUSewer("train","synthetic")#
aau_syn_test =  aau_syn = AAUSewer("test","synthetic")#

aau_real_train =  aau_syn = AAUSewer("train","real")#
aau_real_test =  aau_syn = AAUSewer("test","real")#
    

opt_parser = argparse.ArgumentParser()
opt_parser.add_argument("--epoch",            default = 1000)
opt_parser.add_argument("--lr",               default = 2e-4)
opt_parser.add_argument("--source_batch_size",default = 32)
opt_parser.add_argument("--target_batch_size",default = 32)
opt_parser.add_argument("--warmup_constant",  default = 0.2)
opt_parser.add_argument("--confidence_threshold",default = 0.9)
opt_parser.add_argument("--tau",               default = 0.2)
opt_parser.add_argument("--batch_size",        default = 16)
opt = opt_parser.parse_args(args = [])

/Users/melkor/Desktop/datasets/AAU/training_pointcloud_hdf5_synthetic.h5
/Users/melkor/Desktop/datasets/AAU/testing_pointcloud_hdf5_synthetic.h5
/Users/melkor/Desktop/datasets/AAU/training_pointcloud_hdf5_real.h5
/Users/melkor/Desktop/datasets/AAU/testing_pointcloud_hdf5_real.h5


In [28]:

def DomainCluster(label,feature,c = 0,tau = 0.002):
    cummulate = 0
    size = label.shape[0]
    for i in range(size):
        if int(label[i]) == c:
            contrastive_end = 0
            for j in range(size):
                if int(label[j]) != c:
                    contrastive_end += torch.exp(torch.cosine_similarity(feature[i],feature[j],dim = 0) / tau)
            
            for k in range(size):
                if int(label[i]) == int(label[j]) and i != j:
                    upper = torch.cosine_similarity(feature[i],feature[k],dim = 0) / tau
                    lower = torch.exp(torch.cosine_similarity(feature[i],feature[k],dim = 0) / tau) + contrastive_end
                    cummulate -= upper - torch.log(lower)
                
    return cummulate

def DomainContrast(slabel,tlabel,sfeature,tfeature,c = 0,tau = 0.002):
    cummulate = 0

    s_size = slabel.shape[0]
    t_size = tlabel.shape[0]

    for i in range(s_size):
        if int(slabel[i]) == c:
            contrastive_end = 0
            for j in range(t_size):
                if int(tlabel[j]) != c:
                    contrastive_end += torch.exp(torch.cosine_similarity(sfeature[i],tfeature[j],dim = 0) / tau)
            scores = []
            for j_ in range(t_size):
                if int(slabel[i]) == int(tlabel[j]) and i != j:
                    scores.append(torch.cosine_similarity(sfeature[i],tfeature[j_],dim = 0).detach())
            scores = np.array(scores)
          
            if len(scores) != 0:
                
                k = np.argmax(scores)

                upper = torch.cosine_similarity(sfeature[i],tfeature[k],dim = 0) / tau
                lower = torch.exp(torch.cosine_similarity(sfeature[i],tfeature[k],dim = 0) / tau) + contrastive_end
                cummulate -= upper - torch.log(lower)
    return cummulate

def DomainTransfer(model,opt,source_dataset,target_dataset):

    # create the Adam optimizer with lr
    optimizer = torch.optim.Adam(model.parameters(), lr = opt.lr)

    # setup the dataloader for source domain and target domain

    if not isinstance(source_dataset,DataLoader):
        source_loader = DataLoader(source_dataset,shuffle = True, batch_size = opt.source_batch_size)
    else:source_loader = source_dataset
    if not isinstance(target_dataset,DataLoader):
        target_loader = DataLoader(target_dataset,shuffle = True, batch_size = opt.target_batch_size)
    else:target_loader = target_dataset
    
    # temperature constant tau for scaling
    tau = opt.tau

    for K in range(3): # setup 100 transfer steps
        for _ in range(12): # get this much samples from the source and target dataloader
            working_loss = 0
            try: # get a batch from the source datset 
                source_data, source_label = next(dataloader_iterator)
            except:
                dataloader_iterator = iter(source_loader)
                source_data, source_label = next(dataloader_iterator)
            try: # get a batch from the target dataset
                target_data, target_label = next(target_iterator)
            except:
                target_iterator = iter(target_loader)
                target_data, _ = next(target_iterator)


            # [Calculate the Regular Prediction Loss for Source]
       
            source_prediction,source_feature,_ = model(source_data.permute(0,2,1))
            target_prediction,target_feature,_ = model(target_data.permute(0,2,1))

            # calculate peudo labels for target data

            target_label = np.argmax(target_prediction.cpu().detach().numpy(),1)

            predict_loss = 0


            for i in range(source_prediction.shape[0]):
                predict_loss -= source_prediction[i][source_label[i]]
            working_loss += predict_loss * 500
            
            # [Calculate Contrastive Loss Between Two Batches]
            for clsid in range(4): # peform contrastive learning for binary classification
                # [Source Domain Loss]

                source_loss = DomainCluster(source_label,source_feature,c = clsid)

                # [Target Domain Loss]
                target_loss = DomainCluster(target_label,target_feature,c = clsid)

                # [Inter Domain Loss]
                inter_loss = DomainContrast(source_label,target_label,source_feature,target_feature,c = clsid)
                for loss in [source_loss ,target_loss , inter_loss]:
                    try:
                        working_loss += loss
                    except:pass

            optimizer.zero_grad()
            working_loss.backward()
            optimizer.step()
            
            #print("Working Loss:{} Predict Loss:{}".format(working_loss,predict_loss.detach().numpy()))
        print("K Iter:",K)


In [52]:
def eval(model,dataset):
    if not isinstance(dataset,DataLoader): 
        dataloader = DataLoader(dataset,shuffle = True,batch_size = 4)
    else: dataloader = dataset

    pt_at = 0
    pt_af = 0
    pf_at = 0
    pf_af = 0

    for sample in dataloader:
        x,label = sample

        prediction,feature,_ = model(x.permute(0,2,1))
        for i in range(label.shape[0]):
            predict_label = np.argmax(prediction[i].detach().numpy())
            actual_label = int(label[i])
            if predict_label == 0:
                if actual_label == 0:pt_at += 1
                else:pt_af += 1
            
            if predict_label == 1:
                if actual_label == 0:pf_at += 1
                else:pf_af += 1
    accuracy = (pt_at + pt_af) / (pt_at + pt_af + pf_at + pf_af)
    precision = pt_at/(pt_at + pf_at)
    recall = pt_at/(pt_at + pf_af)
    f1 = 2 * (precision * recall) / (precision + recall )
    print("Raw:{} {} {} {}".format(pt_at,pt_af,pf_at,pf_af))
    print("Actual:{} Precision:{} Recall:{} F1:{} ".format(accuracy,\
                            pt_at/(pt_at + pf_at),\
                            pt_at/(pt_at + pf_af),\
                                f1))

"""
[Setup]
"""

source_dataset = torch.utils.data.ConcatDataset([aau_syn_train,aau_syn_test])
target_dataset = torch.utils.data.ConcatDataset([aau_real_train,aau_real_test])

k = 10
splits=KFold(n_splits=k,shuffle=True,random_state=42)
eval(model,dataset = target_dataset)

Raw:384 375 0 0
Actual:1.0 Precision:1.0 Recall:1.0 F1:1.0 


In [53]:
#next(enumerate(splits.split(np.arange(len(target_dataset)))))

In [54]:
history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}
eval(model,dataset = target_dataset)       
for source_fold, (source_train_idx,source_val_idx) in enumerate(splits.split(np.arange(len(source_dataset)))):
    target_fold, (target_train_idx,target_val_idx) = next(enumerate(splits.split(np.arange(len(target_dataset)))))
    print('Fold {}'.format(source_fold + 1))

    source_train_sampler = SubsetRandomSampler(source_train_idx)
    source_test_sampler = SubsetRandomSampler(source_val_idx)
    source_train_loader = DataLoader(source_dataset, batch_size=opt.batch_size, sampler=source_train_sampler)
    source_test_loader = DataLoader(source_dataset, batch_size=opt.batch_size, sampler=source_test_sampler)
    

    target_train_sampler = SubsetRandomSampler(target_train_idx)
    target_test_sampler = SubsetRandomSampler(target_val_idx)
    target_train_loader = DataLoader(target_dataset, batch_size=opt.batch_size, sampler=target_train_sampler)
    target_test_loader = DataLoader(target_dataset, batch_size=opt.batch_size, sampler=target_test_sampler)
    

    optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

    num_epochs = 1
    for epoch in range(num_epochs):
        DomainTransfer(model,opt,source_train_loader,target_train_loader) 
        eval(model,target_test_loader)       


Raw:384 375 0 0
Actual:1.0 Precision:1.0 Recall:1.0 F1:1.0 
Fold 1
K Iter: 0
K Iter: 1
K Iter: 2
Raw:37 39 0 0
Actual:1.0 Precision:1.0 Recall:1.0 F1:1.0 
Fold 2
K Iter: 0
K Iter: 1
K Iter: 2
Raw:37 39 0 0
Actual:1.0 Precision:1.0 Recall:1.0 F1:1.0 
Fold 3
K Iter: 0
K Iter: 1
K Iter: 2
Raw:37 39 0 0
Actual:1.0 Precision:1.0 Recall:1.0 F1:1.0 
Fold 4
K Iter: 0
K Iter: 1
K Iter: 2
Raw:37 39 0 0
Actual:1.0 Precision:1.0 Recall:1.0 F1:1.0 
Fold 5
K Iter: 0
K Iter: 1
K Iter: 2
Raw:37 39 0 0
Actual:1.0 Precision:1.0 Recall:1.0 F1:1.0 
Fold 6
K Iter: 0
K Iter: 1
K Iter: 2
Raw:37 39 0 0
Actual:1.0 Precision:1.0 Recall:1.0 F1:1.0 
Fold 7
K Iter: 0
K Iter: 1
K Iter: 2
Raw:37 39 0 0
Actual:1.0 Precision:1.0 Recall:1.0 F1:1.0 
Fold 8
K Iter: 0
K Iter: 1
K Iter: 2
Raw:37 39 0 0
Actual:1.0 Precision:1.0 Recall:1.0 F1:1.0 
Fold 9
K Iter: 0
K Iter: 1
K Iter: 2
Raw:37 39 0 0
Actual:1.0 Precision:1.0 Recall:1.0 F1:1.0 
Fold 10
K Iter: 0
K Iter: 1
K Iter: 2
Raw:37 39 0 0
Actual:1.0 Precision:1.0 Recall:1